# Análisis Exploratorio Navent

In [1]:
import datetime as datetime
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
import pyspark
%matplotlib inline
plt.style.use('default') # Make the graphs a bit prettier
plt.rcParams['figure.figsize'] = (15, 5)

In [2]:
import os
path_working_directory = os.getcwd()

educacion = pd.read_csv(os.path.join(path_working_directory, "datos_navent_fiuba", "fiuba_1_postulantes_educacion.csv"), low_memory=False)
genero_edad = pd.read_csv(os.path.join(path_working_directory, "datos_navent_fiuba", "fiuba_2_postulantes_genero_y_edad.csv"), low_memory=False)
visitas = pd.read_csv(os.path.join(path_working_directory, "datos_navent_fiuba", "fiuba_3_vistas.csv"), low_memory=False)
postulaciones = pd.read_csv(os.path.join(path_working_directory, "datos_navent_fiuba", "fiuba_4_postulaciones.csv"), low_memory=False)
avisos_online = pd.read_csv(os.path.join(path_working_directory, "datos_navent_fiuba", "fiuba_5_avisos_online.csv"), low_memory=False)
avisos_detalles = pd.read_csv(os.path.join(path_working_directory, "datos_navent_fiuba", "fiuba_6_avisos_detalle.csv"), low_memory=False)

In [3]:
educacionE6 = pd.read_csv(os.path.join(path_working_directory, "entrega6", "fiuba_1_postulantes_educacion.csv"), low_memory=False)
genero_edadE6 = pd.read_csv(os.path.join(path_working_directory, "entrega6", "fiuba_2_postulantes_genero_y_edad.csv"), low_memory=False)
visitasE6 = pd.read_csv(os.path.join(path_working_directory, "entrega6", "fiuba_3_vistas.csv"), low_memory=False)
postulacionesE6 = pd.read_csv(os.path.join(path_working_directory, "entrega6", "fiuba_4_postulaciones.csv"), low_memory=False)
avisos_onlineE6 = pd.read_csv(os.path.join(path_working_directory, "entrega6", "fiuba_5_avisos_online.csv"), low_memory=False)
avisos_detallesE6 = pd.read_csv(os.path.join(path_working_directory, "entrega6", "fiuba_6_avisos_detalle.csv"), low_memory=False)

In [4]:
educacion = educacion.append(educacionE6)
genero_edad = genero_edad.append(genero_edadE6)
visitas = visitas.append(visitasE6)
postulaciones = postulaciones.append(postulacionesE6)
avisos_online = avisos_online.append(avisos_onlineE6)
avisos_detalles = avisos_detalles.append(avisos_detallesE6)

In [5]:
educacion15A = pd.read_csv(os.path.join(path_working_directory, "Fiuba desde 15 Abril", "fiuba_1_postulantes_educacion.csv"), low_memory=False)
genero_edad15A = pd.read_csv(os.path.join(path_working_directory, "Fiuba desde 15 Abril", "fiuba_2_postulantes_genero_y_edad.csv"), low_memory=False)
visitas15A = pd.read_csv(os.path.join(path_working_directory, "Fiuba desde 15 Abril", "fiuba_3_vistas.csv"), low_memory=False)
avisos_detalles15A = pd.read_csv(os.path.join(path_working_directory, "datos_navent_fiuba", "fiuba_6_avisos_detalle.csv"), low_memory=False)

Primero veremos algunas filas de cada uno de los dataframes levantados

In [6]:
educacion.head(5)

,idpostulante,nombre,estado
0,NdJl,Posgrado,En Curso
1,8BkL,Universitario,En Curso
2,1d2B,Universitario,En Curso
3,NPBx,Universitario,En Curso
4,NPBx,Master,En Curso


In [7]:
genero_edad.head(5)

,idpostulante,fechanacimiento,sexo
0,NM5M,1970-12-03,FEM
1,5awk,1962-12-04,FEM
2,ZaO5,1978-08-10,FEM
3,NdJl,1969-05-09,MASC
4,eo2p,1981-02-16,MASC


In [8]:
visitas.head(5)

,idAviso,timestamp,idpostulante
0,1111780242,2018-02-23T13:38:13.187-0500,YjVJQ6Z
1,1112263876,2018-02-23T13:38:14.296-0500,BmVpYoR
2,1112327963,2018-02-23T13:38:14.329-0500,wVkBzZd
3,1112318643,2018-02-23T13:38:17.921-0500,OqmP9pv
4,1111903673,2018-02-23T13:38:18.973-0500,DrpbXDP


In [9]:
postulaciones.head(5)

,idaviso,idpostulante,fechapostulacion
0,1112257047,NM5M,2018-01-15 16:22:34
1,1111920714,NM5M,2018-02-06 09:04:50
2,1112346945,NM5M,2018-02-22 09:04:47
3,1112345547,NM5M,2018-02-22 09:04:59
4,1112237522,5awk,2018-01-25 18:55:03


In [10]:
avisos_online.head(5)

,idaviso
0,1112355872
1,1112335374
2,1112374842
3,1111984070
4,1111822480


In [11]:
avisos_detalles.head(5)

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,8725750,1,VENDEDOR/A PROVINCIA DE SANTA FE,<p><strong><strong>Empresa:</strong></strong> ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,VENTOR
1,17903700,1,Enfermeras,<p>Solicitamos para importante cadena de farma...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Salud,Farmacias Central Oeste
2,1000150677,1,Chofer de taxi,<p>TE GUSTA MANEJAR? QUERES GANAR PLATA HACIEN...,Capital Federal,NaN,Empedrado 2336,Full-time,Senior / Semi-Senior,Transporte,FAMITAX SRL
3,1000610287,1,CHOFER DE CAMIONETA BAHIA BLANCA - PUNTA ALTA,<p><strong>Somos una empresa multinacional que...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Transporte,Wurth Argentina S.A
4,1000872556,1,Operarios de Planta - Rubro Electrodomésticos,<p><strong>OPERARIOS DE PLANTA</strong></p><p>...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL


## Limpieza de los set de datos

### Set de genero y fechas de nacimiento

In [12]:
print "Cantidad de usuarios:", len(genero_edad), '\n'
print "Los usuarios son unicos:", len(genero_edad) == len(genero_edad.groupby('idpostulante')), '\n'
print genero_edad.isnull().any(), '\n'
print genero_edad.isnull().sum(), '\n'
print genero_edad.dtypes, '\n'

Cantidad de usuarios: 498635 

Los usuarios son unicos: False 

idpostulante       False
fechanacimiento     True
sexo               False
dtype: bool 

idpostulante           0
fechanacimiento    21370
sexo                   0
dtype: int64 

idpostulante       object
fechanacimiento    object
sexo               object
dtype: object 



Como se puede observar, existen 4750 usuarios de 200888 que no declaran su fecha de nacimiento (es decir, aproximadamente un 2,36%). Para que el set de datos esté completo se podría rellenar los datos faltantes realizando una regresión o sacando un promedio siguiendo algún tipo de criterio. Sin embargo, como la cantidad de usuarios en cuestión es baja, se los puede eliminar del set de datos. Antes de realizar esto se analizará cuántas postulaciones y visitas corresponden a usuarios sin fecha de nacimiento, para ver si eliminar dichos usuarios implica quitar muchas postulaciones y visitas a avisos de los demas sets de datos.

In [13]:
no_declaran_edad = genero_edad[genero_edad.fechanacimiento.isnull()]
print len(no_declaran_edad), "postulantes no declaran edad", '\n'
declaran_edad = genero_edad[genero_edad.fechanacimiento.notnull()]
print len(declaran_edad), "postulantes declaran edad", '\n'
postulaciones_sin_fecha_nacimiento = pd.merge(no_declaran_edad, postulaciones, how='inner', on='idpostulante')
print len(postulaciones_sin_fecha_nacimiento), "postulaciones de usuarios sin fecha de nacimiento de", len(postulaciones), '\n'
visitas_sin_fecha_nacimiento = pd.merge(no_declaran_edad, visitas, how='inner', on='idpostulante')
print len(visitas_sin_fecha_nacimiento), "visitas de usuarios sin fecha de nacimiento de", len(visitas)

21370 postulantes no declaran edad 

477265 postulantes declaran edad 

185442 postulaciones de usuarios sin fecha de nacimiento de 8311264 

234751 visitas de usuarios sin fecha de nacimiento de 6720583


Aproximadamente un 1,31% de las postulaciones corresponden a usuarios sin fecha de nacimiento mientras que en el caso de las visitas es de un 2,37%. Teniendo en cuenta que estos valores son pequeños, podemos optar por eliminar los usuarios sin fecha de nacimiento (y hacer lo propio con sus postulaciones y visitas a avisos).

In [14]:
genero_edad_filtrado = genero_edad[genero_edad.fechanacimiento.notnull()]

In [15]:
len(genero_edad_filtrado)

477265

### Set de estudios

In [16]:
print educacion.isnull().any(), '\n'
print educacion.describe(include='all'), '\n'
print educacion.dtypes, '\n'
print educacion.nombre.value_counts(), '\n'
print educacion.estado.value_counts(), '\n'

idpostulante    False
nombre          False
estado          False
dtype: bool 

       idpostulante         nombre    estado
count        705684         705684    705684
unique       366395              7         3
top          YlMLGD  Universitario  Graduado
freq             18         259550    460482 

idpostulante    object
nombre          object
estado          object
dtype: object 

Universitario        259550
Secundario           246012
Terciario/Técnico    114815
Otro                  53513
Posgrado              20885
Master                10297
Doctorado               612
Name: nombre, dtype: int64 

Graduado      460482
En Curso      184467
Abandonado     60735
Name: estado, dtype: int64 



Se puede observar que en algunos casos existen múltiples entradas de estudio por usuario. A continuación se analizará el caso del usuario que tiene más entradas, cuyo id es YlMLGD:

In [17]:
educacion[educacion['idpostulante'] == 'YlMLGD']

,idpostulante,nombre,estado
373,YlMLGD,Terciario/Técnico,En Curso
374,YlMLGD,Otro,En Curso
375,YlMLGD,Universitario,En Curso
2206,YlMLGD,Terciario/Técnico,Graduado
2207,YlMLGD,Universitario,Graduado
2208,YlMLGD,Otro,Graduado
2209,YlMLGD,Posgrado,Graduado
3659,YlMLGD,Universitario,Abandonado
3660,YlMLGD,Terciario/Técnico,Abandonado
354900,YlMLGD,Terciario/Técnico,En Curso


Podemos ver que este usuario presenta múltiples entradas con estudios con estado En Curso, Graduado y Abandonado. A su vez, aparencen múltiples entradas para un mismo tipo de estudio, como es el caso de Universitario.
Para poder analizar mejor los datos de educación, se optó por generar un dataframe que tenga una fila por cada usuario donde sus columnas sean el id del usuario (idpostulante), el mayor nivel de estudio en el que se graduó (estudio_graduado) y el mayor nivel de estudio que tiene en curso (estudio_en_curso), que a su vez debe ser mayor al nivel que tiene en la columna estudio_graduado.
Se definió que el nivel de estudio, de menor a mayor, sea el siguiente: 
Otro, Secundario, Terciario/Técnico, Universitario, Master, Posgrado y Doctorado.

In [18]:
tipos_de_estudio = ['Ninguno', 'Otro', 'Secundario', 'Terciario/Técnico', 'Universitario', 'Master', 'Posgrado', 'Doctorado']
def get_dict_tipos_de_estudio():
    dict = {}
    for x in range(len(tipos_de_estudio)):
        dict[tipos_de_estudio[x]] = x
    return dict

dict_tipos_de_estudio = get_dict_tipos_de_estudio()

def mejor_estudio(estudio1, estudio2):
    return tipos_de_estudio[max(tipos_de_estudio.index(estudio1), tipos_de_estudio.index(estudio2))]

en_curso = educacion.apply(lambda x: dict_tipos_de_estudio[x['nombre']] if x['estado'] == 'En Curso' else 0, axis=1)
graduado = educacion.apply(lambda x: dict_tipos_de_estudio[x['nombre']] if x['estado'] == 'Graduado' else 0, axis=1)
educacion2 = educacion.assign(estudio_en_curso=en_curso)
educacion2 = educacion2.assign(estudio_graduado=graduado)

In [19]:
educacion_grupos = educacion2.groupby('idpostulante')
educacion_nueva = pd.DataFrame({
    'estudio_en_curso': educacion_grupos['estudio_en_curso'].max(),
    'estudio_graduado': educacion_grupos['estudio_graduado'].max()
}).reset_index()
educacion_nueva['estudio_en_curso'] = educacion_nueva.apply(lambda x: tipos_de_estudio[x['estudio_en_curso']] if x['estudio_en_curso'] > x['estudio_graduado'] else 'Ninguno', axis=1)
educacion_nueva['estudio_graduado'] = educacion_nueva['estudio_graduado'].apply(lambda x: tipos_de_estudio[x])
print len(educacion_nueva)
educacion_nueva[educacion_nueva['idpostulante'] == 'YlMLGD']

366395


,idpostulante,estudio_en_curso,estudio_graduado
228953,YlMLGD,Ninguno,Posgrado


El usuario YlMLGD no tiene una entrada en la columna de estudio en curso dado que esta graduado en un Posgrado (y su mayor estudio en curso es Universitario, que tiene un menor nivel). 
De esta manera, se tiene una única entrada por cada usuario y la cantidad de entradas se corresponde con la cantidad de valores únicos en el dataframe original.

Solamente queremos mantener usuarios que tengan informacion sobre su educación:

In [20]:
usuarios_educacion_edad = pd.merge(genero_edad_filtrado, educacion_nueva, on='idpostulante', how='inner')
len(usuarios_educacion_edad)

444083

In [21]:
postulaciones.head()

,idaviso,idpostulante,fechapostulacion
0,1112257047,NM5M,2018-01-15 16:22:34
1,1111920714,NM5M,2018-02-06 09:04:50
2,1112346945,NM5M,2018-02-22 09:04:47
3,1112345547,NM5M,2018-02-22 09:04:59
4,1112237522,5awk,2018-01-25 18:55:03


Dentro de los usuarios con entradas de educación, la cantidad de usuarios sin fecha de nacimiento no es alta (2874 de 188752, que es un 1,52%) y dado que se quería realizar un análisis teniendo en cuenta esta información, se optó por borrarlos.

In [22]:
postulaciones_total = len(postulaciones)
postulaciones_filtrado = pd.merge(usuarios_educacion_edad, postulaciones, how='inner', on='idpostulante')
print len(postulaciones_filtrado), "postulaciones de usuarios con fecha de nacimiento y educacion de", postulaciones_total, '\n'
visitas_total = len(visitas)
visitas_filtrado = pd.merge(usuarios_educacion_edad, visitas, how='inner', on='idpostulante')
print len(visitas_filtrado), "visitas de usuarios con fecha de nacimiento y educacion de", visitas_total

13017308 postulaciones de usuarios con fecha de nacimiento y educacion de 8311264 

9409417 visitas de usuarios con fecha de nacimiento y educacion de 6720583


In [23]:
visitas_filtrado.rename(index=str, columns={'idAviso': 'idaviso'}, inplace=True)
visitas_filtrado.head(1)

,idpostulante,fechanacimiento,sexo,estudio_en_curso,estudio_graduado,idaviso,timestamp
0,5awk,1962-12-04,FEM,Ninguno,Universitario,1112315188,2018-02-28T15:53:48.610-0500


In [24]:
avisos_detalles.drop('ciudad', axis=1, inplace=True)
avisos_detalles.drop('mapacalle', axis=1, inplace=True)

In [25]:
avisos_sin_empresa = avisos_detalles[avisos_detalles.denominacion_empresa.isnull()]
postulaciones_avisos_sin_empresa = pd.merge(postulaciones_filtrado, avisos_sin_empresa, on='idaviso', how='inner')
visitas_avisos_sin_empresa = pd.merge(visitas_filtrado, avisos_sin_empresa, on='idaviso', how='inner')

In [26]:
avisos_detalles = avisos_detalles[avisos_detalles.denominacion_empresa.notnull()]
postulaciones_filtrado = postulaciones_filtrado[~postulaciones_filtrado.idaviso.isin(avisos_sin_empresa.idaviso)]
visitas_filtrado = visitas_filtrado[~visitas_filtrado.idaviso.isin(avisos_sin_empresa.idaviso)]
print len(avisos_online)
avisos_online = avisos_online[~avisos_online.idaviso.isin(avisos_sin_empresa.idaviso)]
print len(avisos_online)

9886
9886


# Prediccion

In [27]:
def aEnumNivel(nivel):
    if(nivel == 'Gerencia / Alta Gerencia / Dirección'):
        return 0
    if(nivel == 'Jefe / Supervisor / Responsable'):
        return 1
    if(nivel == 'Junior'):
        return 2
    if(nivel == 'Senior / Semi-Senior'):
        return 3
    return 4

def aEnumTipo(nivel):
    if(nivel == 'Full-time'):
        return 0
    if(nivel == 'Part-time'):
        return 1
    if(nivel == 'Teletrabajo'):
        return 2
    if(nivel == 'Pasantia'):
        return 3
    if(nivel == 'Por Horas'):
        return 4
    if(nivel == 'Temporario'):
        return 5
    if(nivel == 'Por Contrato'):
        return 6
    if(nivel == 'Fines de Semana'):
        return 7
    return 8

In [28]:
areas = avisos_detalles['nombre_area'].values

In [29]:
def aEnumArea(area):
    for x in range (0, len(areas)):
        if area == areas[x]:
            return x

In [30]:
avisos_detalles.drop('idpais', axis=1, inplace=True)
avisos_detalles.drop('nombre_zona', axis=1, inplace=True)
avisos_detalles['tipo_de_trabajo'] = avisos_detalles['tipo_de_trabajo'].map(lambda x: aEnumTipo(x))
avisos_detalles['nivel_laboral'] = avisos_detalles['nivel_laboral'].map(lambda x: aEnumNivel(x))
avisos_detalles['nombre_area'] = avisos_detalles['nombre_area'].map(lambda x: aEnumArea(x))

In [31]:
avisos_detalles.head(1)

,idaviso,titulo,descripcion,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,8725750,VENDEDOR/A PROVINCIA DE SANTA FE,<p><strong><strong>Empresa:</strong></strong> ...,0,3,0,VENTOR


In [32]:
def completoAEnum(completo):
    estudio = ['Ninguno', 'Otro', 'Secundario', 'Terciario/Técnico', 'Universitario', 'Master', 'Posgrado', 'Doctorado']
    for x in range(0,len(estudio)):
        if completo == estudio[x]:
            return x
    return 0

In [33]:
postulaciones_filtrado.drop('fechanacimiento', axis = 1, inplace=True)
postulaciones_filtrado.drop('fechapostulacion', axis = 1, inplace=True)
postulaciones_filtrado['estudio_en_curso'] = postulaciones_filtrado['estudio_en_curso'].apply(lambda x: completoAEnum(x))
postulaciones_filtrado['estudio_graduado'] = postulaciones_filtrado['estudio_graduado'].apply(lambda x: completoAEnum(x))
postulaciones_filtrado.head(1)

,idpostulante,sexo,estudio_en_curso,estudio_graduado,idaviso
0,NM5M,FEM,0,2,1112257047


In [34]:
postulaciones_filtrado.head(1)

,idpostulante,sexo,estudio_en_curso,estudio_graduado,idaviso
0,NM5M,FEM,0,2,1112257047


In [35]:
visitas_filtrado.drop('fechanacimiento', axis = 1, inplace=True)
visitas_filtrado.drop('sexo', axis = 1, inplace=True)
visitas_filtrado.drop('estudio_en_curso', axis = 1, inplace=True)
visitas_filtrado.drop('estudio_graduado', axis = 1, inplace=True)
visitas_filtrado.drop('timestamp', axis = 1, inplace=True)
visitas_filtrado['acum'] = 1

In [36]:
total_visitas = pd.DataFrame(visitas_filtrado.groupby(['idpostulante', 'idaviso']).agg('sum').reset_index().as_matrix())

/home/gonzalo/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [37]:
total_visitas.columns=['idpostulante', 'idaviso', 'cant_visitas']

In [38]:
postutalles = pd.merge(postulaciones_filtrado, avisos_detalles, on='idaviso', how='inner')

In [39]:
total_visitas['idaviso'] = total_visitas['idaviso'].apply(lambda x: int(x))

In [40]:
postutalles = pd.merge(postutalles, total_visitas, how='left', left_on=['idaviso', 'idpostulante'], right_on=['idaviso', 'idpostulante'])

In [41]:
postutalles['cant_visitas'] = postutalles['cant_visitas'].apply(lambda x: 0 if pd.isna(x) else x)

# Comienzo de uso de algoritmos

In [43]:
def chequearExistencia(aviso, postulante):
    postutalles.loc[(postutalles['idaviso'] == aviso) & (postutalles['idpostulante'] == 'postulante')].shape == (0,12)

In [44]:
#postutalles.shape
postutalles.shape

(21093283, 13)

In [45]:
postSample = postulaciones_filtrado.sample(int(21093283*0.4), replace=True)
avisSample = avisos_detalles.sample(int(21093283*0.4), replace=True)
postSample['idaviso'] = avisSample['idaviso'].tolist()
postSample['titulo'] = avisSample['titulo'].tolist()
postSample['descripcion'] = avisSample['descripcion'].tolist()
postSample['tipo_de_trabajo'] = avisSample['tipo_de_trabajo'].tolist()
postSample['nivel_laboral'] = avisSample['nivel_laboral'].tolist()
postSample['nombre_area'] = avisSample['nombre_area'].tolist()
postSample['denominacion_empresa'] = avisSample['denominacion_empresa'].tolist()
postSample['cant_visitas'] = 0

In [48]:
postutalles['se_postulo'] = 1
postSample['se_postulo'] = 0
print(postutalles.shape)
print(postSample.shape)

(21093283, 13)
(8437313, 13)


In [51]:
postutalles = postutalles.append(postSample)
print(postutalles.shape)

(29530596, 13)


In [52]:
postutalles.head(1)

,idpostulante,sexo,estudio_en_curso,estudio_graduado,idaviso,titulo,descripcion,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cant_visitas,se_postulo
0,NM5M,FEM,0,2,1112257047,REPRESENTANTES DE ATENCIÓN AL CLIENTE/ RETENCIÓN,"<p><strong><em><span style="""">En </span></em><...",1,4,9,Eficasia Argentina SA,0,1


In [86]:
pd.merge(visitas, avisos_detalles, how='left', right_on='idaviso', left_on='idAviso').shape

(9093283, 10)

In [90]:
test = pd.read_csv(os.path.join(path_working_directory, "datos_navent_fiuba", "test_final_100k.csv"), low_memory=False)

In [56]:
toTrain.head(1)

,idpostulante,sexo,estudio_en_curso,estudio_graduado,idaviso,titulo,descripcion,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cant_visitas,se_postulo
2491232,BmDa4Yw,MASC,0,2,1112230537,Jefe Administrativo,<p>Para Importante Empresa Industrial</p><p>Se...,0,1,19,Manpower,0,1


In [57]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [55]:
toTrain = postutalles.sample(29530596 / 20)
train_y = toTrain['se_postulo']
train_x = toTrain.drop('se_postulo', axis=1)
#clf = RandomForestClassifier(max_depth=2, random_state=0)
#clf.fit(train_x, train_y)

ValueError: could not convert string to float: Pop Salado

In [61]:
from sklearn import tree

In [62]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(train_x, train_y)

ValueError: could not convert string to float: Pop Salado

In [63]:
train_x

,idpostulante,sexo,estudio_en_curso,estudio_graduado,idaviso,titulo,descripcion,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cant_visitas
2491232,BmDa4Yw,MASC,0,2,1112230537,Jefe Administrativo,<p>Para Importante Empresa Industrial</p><p>Se...,0,1,19,Manpower,0
18772005,dYjArPq,MASC,0,2,1112244070,Repositores para Capital Federal y GBA,"<p>Manpower Argentina, se encuentra en la búsq...",0,3,18,Manpower,0
13854791,jkVGXNQ,MASC,0,4,1112423753,Responsable de Procesos,"<p> </p><p style=""""><span style="""">El desafío ...",0,1,159,Cerro Vanguardia,8
9444899,6rPBMNv,NO_DECLARA,0,4,1112338408,Pasante Mejora Continua,<p>¿Estás buscando una oportunidad desafiante ...,1,2,574,Pepsico de Argentina SRL,0
7585742,vVDdMEo,FEM,0,3,1112346678,CUSTODIOS QUE RESIDAN EN ZONA NORTE,<p><strong>IMPORTANTE EMPRESA DE SEGURIDAD se...,0,3,176,Work Solutions S.R.L.,0
18808024,xkPoWJP,FEM,0,2,1112340028,OPERARIA DE PRODUCCION,<p><strong>Importante empresa de Logística</st...,0,2,4,Asoko Tempo SA,0
6563374,0zLep5v,MASC,4,2,1111209509,EMPLEADO PARA FABRICA PIRKA STONE,<p>Se busca empleado para fabrica de revestimi...,0,4,4,PIRKA STONE,0
10015984,Rzr39Gq,FEM,0,2,1112398171,RECEPCIONISTA - ASISTENTE,<p>Nos encontramos en la búsqueda de una recep...,0,2,123,Varennes Corp,0
10595162,JBrGQwe,FEM,0,2,1112306501,Procurement analyst (PSC),<p>The role is to work within a BPO environmen...,0,3,134,SDI International,0
17384926,dYoVpXq,MASC,4,2,1112035206,Analista Contable Junior - para Importante Em...,<p>Nos encontramos en la búsqueda de un <stron...,0,2,20,Gentile y Asociados Consultores,0


In [105]:
#cuantos usuarios vieron un anuncio
visitas['idpostulante'].value_counts()

qe2Bx4N    6565
8aVPVO     2616
8MaQjP3    2565
96LklB5    2018
dYoPxP2    1602
8MPQpBz    1505
6rZokLv    1481
JBderlr    1370
0zBDZRq    1233
owjzK8W    1230
QNaJPVW    1119
PmB8DP0    1106
1pY05B     1101
W9kYGYm    1086
NZYA05     1078
YQb6mr     1065
1O9O4j     1064
aMpBwZ     1035
61wJQL     1017
KBzm0ZX    1017
pzeMmbp    1012
EBz3Bo     1009
Rz9E58Y     969
8KorYj      964
A3AR531     947
YjrAvRq     947
kPjkEGm     942
Ez0LdL9     929
BmB0qVa     929
5mP3b5p     924
           ... 
VNr1902       1
96XeOR5       1
JBobPZe       1
6KVQaj        1
YVdOmY        1
Eexm12        1
YjQPw1r       1
KBeOW00       1
mzbox39       1
KBaP6KX       1
0zdZvbd       1
bOjrqbE       1
EzeQY1N       1
65M40x        1
BmB34wZ       1
wVk188w       1
dYjNokX       1
MVPOKBN       1
W9AXjjB       1
OqdDDed       1
96MK4Gp       1
lDwbY02       1
N6rjPJ        1
MVzj0bL       1
NzrZNwV       1
ZPjEwY        1
11Z4BE        1
96ZJj2p       1
1QPlNxR       1
kPjBD91       1
Name: idpostulante, Leng

In [106]:
genero_edad.shape

(498635, 3)

In [107]:
#se postulo a un anuncio
postulaciones['idpostulante'].value_counts()

8MaQjP3    4932
5Mwjak     3988
6rQdqjl    3891
KBzm0ZX    3565
axmjOE     3166
ZD2081P    3068
ZQZJJb     2821
aWY5D5     2729
ow2oqjW    2610
5mraqEZ    2598
Ejb554     2423
akDVNzJ    2395
5354Mw     2202
vVjZ16P    2163
4rPx4jR    2125
EzeELAz    2081
qevreVR    2029
OqrGRox    1844
Oqre00r    1828
0zBDZRq    1800
vVDd60Q    1800
pzeb1jm    1758
8MPw0mO    1745
xkv48Lb    1742
rmRMzNz    1696
8MBBqqW    1690
4rPOqMq    1666
QNEWDAN    1619
W9kYGYm    1574
1QPL55N    1564
           ... 
5mr4Baw       1
LNPp6Y5       1
8MPdqEj       1
5mPvO3k       1
QNreB04       1
akj3Mv5       1
RzP95WR       1
Nz0z1Y5       1
VNxakxD       1
2zmE3dY       1
e4R2N4        1
lD2D3N2       1
lD5Zzz8       1
ow2rDBW       1
OqPj0ld       1
jk5zq6e       1
8MkEAz        1
rm0wWjA       1
PmG6BQR       1
LN8EDPq       1
1QP3p8z       1
4rPkaDK       1
ekOVZbr       1
owjp3JL       1
mzbY4eE       1
xkdomjP       1
lDLAljM       1
LNKpMO5       1
xkd1ja0       1
ZDPVkwb       1
Name: idpostulante, Leng

In [109]:
#hicieron ambas
pd.merge(postulaciones, visitas, how='inner', left_on=['idaviso', 'idpostulante'], right_on=['idAviso', 'idpostulante'])['idpostulante'].value_counts()


qe2Bx4N    6529
8MaQjP3    2436
8aVPVO     2157
dYoPxP2    1331
6rZokLv    1317
0zBDZRq    1309
JBderlr    1295
W9kYGYm    1266
owjzK8W    1211
KBzm0ZX    1102
EBz3Bo     1063
1pY05B     1006
YjrAvRq     980
pzeMmbp     980
A3AR531     975
OqrGRox     909
ZD2081P     884
aMpBwZ      882
5mP3b5p     878
5vDEvX      877
akDVNzJ     871
61wJQL      869
OqrXMmv     861
kPjkEGm     820
Ejb554      815
5mqz2Wq     796
Rz9E58Y     796
VNkl3NR     786
VNrRKJd     781
aWY5D5      769
           ... 
lDbx9Nz       1
xkvAAYa       1
YmBxQD        1
ZDlqdX8       1
EzpQ6A4       1
JBeqbe3       1
vVj1BVQ       1
QNaLYvo       1
NzroAwD       1
ZD8O9mz       1
mzbYkBG       1
VNrWQvM       1
5KOr1q        1
96zzmZ8       1
ZYAem5        1
2zmWl2Y       1
4rPrJvq       1
MVdz50N       1
QNE5xXN       1
1pDb          1
W9V14L2       1
3Nkplaq       1
MVrXEbv       1
BmBd0E0       1
akjZL9E       1
N5qAkB        1
DrJQqdK       1
YjNb2vq       1
GNZGRjK       1
ZD8rJqV       1
Name: idpostulante, Leng